In [ ]:
import numpy as np
import torch
import pyepo

### Number of Cores 

In [ ]:
# number of cores
grids = [(5,5), (8,8)]#, (10,10), (12,12), (15,15), (18,18), (20,20)]

### Data 

In [ ]:
from dataset import optDatasetConstrs
from torch.utils.data import DataLoader

def getDataLoader(grid):
    # generate data
    num_data = 1000 # number of training data
    num_feat = 5 # size of feature
    deg = 4 # polynomial degree
    e = 0.5 # noise width
    feats, costs = pyepo.data.shortestpath.genData(num_data, num_feat, grid, deg, e, seed=42)
    # create dataset
    dataset = optDatasetConstrs(optmodel, feats, costs=costs)
    # get data loader
    batch_size = 32
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
    return dataloader

### Linear Regression 

In [ ]:
import torch
from torch import nn

# build linear model
class LinearRegression(nn.Module):

    def __init__(self):
        super(LinearRegression, self).__init__()
        self.linear = nn.Linear(num_feat, (grid[0]-1)*grid[1]+(grid[1]-1)*grid[0])

    def forward(self, x):
        out = self.linear(x)
        return out

### Plot

In [ ]:
from matplotlib import pyplot as plt
import tol_colors as tc

def plot(grids, elapsed_grb, elapsed_clb, elapsed_nnls, title):
    sizes = [g[0] for g in grids]
    # color map
    cset =  tc.tol_cset('light')
    fig = plt.figure(figsize=(16,8))
    plt.plot(sizes, elapsed_grb, marker="o", linestyle="--", markersize=10, lw=5, color=cset.orange, label="Gurobi")
    plt.plot(sizes, elapsed_clb, marker="o", linestyle="--", markersize=10, lw=5, color=cset.light_blue, label="Clarabel")
    plt.plot(sizes, elapsed_nnls, marker="o", linestyle="--", markersize=10, lw=5, color=cset.mint, label="NNLS")
    plt.xlabel("Size of Grid", fontsize=36)
    plt.ylabel("Runtime per Epoch (Sec)", fontsize=36)
    plt.xticks(ticks=["{}×{}".format(g[0], g[1]) for g in grids], fontsize=22)
    plt.yticks(fontsize=28)
    plt.legend(fontsize=32)
    plt.title(title, fontsize=36)
    plt.show()

### Training 

In [ ]:
import copy
import time
from tqdm import tqdm

from pyepo.model.grb import shortestPathModel

from func import exactConeAlignedCosine

def pipeline():
    elapsed_grb, elapsed_clb, elapsed_nnls = [], [], []
    for grid in grids:
        # set random seed
        np.random.seed(42)
        torch.manual_seed(42)
        # set solver
        optmodel = shortestPathModel(grid)
        # init predictor
        reg = LinearRegression(grid)
        # get dataloader
        dataloader = getDataLoader(grid)
        # gurobi
        time.sleep(1)
        print("Gurobi:")
        elapsed = trainingProcess(reg, optmodel, "gurobi", dataloader)
        elapsed_grb.append(elapsed)
        # clarabel
        time.sleep(1)
        print("Clarabel:")
        elapsed = trainingProcess(reg, optmodel, "clarabel", dataloader)
        elapsed_clb.append(elapsed)
        # nnls
        time.sleep(1)
        print("NNLS:")
        elapsed = trainingProcess(reg, optmodel, "nnls", dataloader)
        elapsed_nnls.append(elapsed)
    return elapsed_grb, elapsed_clb, elapsed_nnls


def trainingProcess(reg, optmodel, solver, dataloader):
    # copy predictor
    reg = copy.deepcopy(reg)
    # init loss
    ca_cos = exactConeAlignedCosine(optmodel, solver=solver)
    # train
    elapsed = train(reg, ca_cos, dataloader)
    return elapsed

        
def train(reg, loss_func, dataloader):
    # set optimizer
    optimizer = torch.optim.Adam(reg.parameters(), lr=5e-2)
    # training
    tbar = tqdm(dataloader)
    tick = time.time()
    for data in tbar:
        # forward pass
        x, _, _, t_ctr = data
        # predict
        cp = reg(x)
        # loss
        loss = loss_func(cp, t_ctr)
        # backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        tbar.set_description("Loss: {:8.4f}".format(loss.item()))
    # record time
    tock = time.time()
    elapsed = tock - tick
    return elapsed

In [ ]:
elapsed_grb, elapsed_clb, elapsed_nnls = pipeline(reg, optmodel, loader_train)

In [ ]:
plot(sizes, elapsed_grb, elapsed_clb, elapsed_nnls, "Shortest Path")